# Pytorch Lightning
#### Experiments using Pytorch Lightning to build custom CNN classification model.


Pytorch Lightning implementation
In this notebook I will implement CNN model using Pytorch Lightning. This model will be more flexible, than model from initial_experiments.ipynb, to provide more hyperparameters for training sessions.

In [3]:
from scripts.models.lightning_model import LightningCNN, ImageDataModule

In [4]:
import os
from pathlib import Path

from dotenv import load_dotenv
load_dotenv()
root_data = os.getenv("KAGGLE_FILES_DIR")
dataset_path = Path(os.getcwd(), "..", root_data, 'processed')

Transformations by default are only resizing to 256x256.

In [5]:
datamodule = ImageDataModule(data_dir=dataset_path, batch_size=32)

# Training
1st model: default parameters:
- conv_layers: 5
- fc_layer_sizes: (256, 128)
- input_size: torch.Size([3, 256, 256])
- out_classes: 1
- initial_filters: 32
- hl_kernel_size: 5
- activation_func: nn.ReLU
- max_pool_kernel: 2
- dropout_conv: False
- dropout_fc: False
- dropout_rate: 0.5
- initial_learning_rate: 0.01
- loss_func: nn.BCEWithLogitsLoss
- optimizer: Adam
- metrics: Accuracy, Precision, Recall, F1, AUC, ConfusionMatrix

In [8]:
import torch
import pytorch_lightning as pl

model_1 = LightningCNN(
    conv_layers=5,
    fc_layer_sizes=(256, 128),
    input_size=torch.Size([3, 256, 256]),
    initial_filters=32,
    out_classes=1,
    hl_kernel_size=5,
    max_pool_kernel=2,
    dropout_conv=False,
    dropout_fc=False,
    initial_learning_rate=0.01
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_loss',
    dirpath='../models/lightning',
    filename='model-9M-5conv-nodrop-{epoch:02d}-{valid_loss:.2f}',
    save_top_k=2,
    mode='min') 

early_stopping = pl.callbacks.EarlyStopping(
    monitor='valid_loss',
    min_delta=0.001,
    patience=5,
    verbose=True,
    mode='min'
)

trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, early_stopping]
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model_1, datamodule)


  | Name          | Type                  | Params
--------------------------------------------------------
0 | loss_func     | BCEWithLogitsLoss     | 0     
1 | accuracy      | BinaryAccuracy        | 0     
2 | precision     | BinaryPrecision       | 0     
3 | recall        | BinaryRecall          | 0     
4 | f1            | BinaryF1Score         | 0     
5 | auc           | BinaryAUROC           | 0     
6 | confmat       | BinaryConfusionMatrix | 0     
7 | hidden_layers | Sequential            | 4.4 M 
8 | fc_layers     | Sequential            | 4.8 M 
--------------------------------------------------------
9.1 M     Trainable params
0         Non-trainable params
9.1 M     Total params
36.429    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]